In [1]:
%reset

from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
from Crypto.Random import get_random_bytes
from base64 import b16encode
from base64 import b16decode
import os
import json
import hmac
import hashlib

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Initialize Data
data = b"Accnt:0003336669Accnt:0123456789Descr:0000000000000000000000000000000000000000000000000000000000Amount:023456789"
key = get_random_bytes(16)
print(b16encode(key).decode('utf-8'))

D5C1B08852BE1B16826CE899767449D7


# Task D

In [3]:
def xor(A, B):
    return hex(int(A, 16) ^ int(B, 16))[2:].upper() 

In [4]:
#Like Task B's attack
def attack(data):
	b16 = json.loads(data)
	ciphertext = b16['ct']
	iv = b16['iv']
	block_to_alter = ciphertext[160:192] 
	byte_to_change = block_to_alter[14:16]
	
	m = xor('0x30', '0x39')
	
	#perform bit flipping
	new_byte = xor(hex(int(m, 16)), hex(int(byte_to_change, 16)))
	while len(new_byte) < 2:
	    new_byte = "".join(['0', new_byte])
	new_ciphertext = ''.join([ciphertext[0:174], new_byte, ciphertext[176:]]) 
	
	result = json.dumps({'iv':iv, 'ct':new_ciphertext})
	return result

In [12]:
#Like Task C's Decryption
def decrypt_message(encrypted_data, iv_bytes, signature, shared_key, hmac_key):
    iv = iv_bytes + encrypted_data
    new_hmac = hmac.new(hmac_key, iv, hashlib.sha256).digest()
    if not compare_mac(new_hmac, signature):
    	print("Attack!")
    cypher = AES.new(shared_key, AES.MODE_CBC, iv_bytes)
    plaintext = cypher.decrypt(encrypted_data)
    return unpad(plaintext, AES.block_size)

In [7]:
def compare_mac(a, b):
    a = a[1:] 
    b = b[1:]
    different = 0
    
    for x, y in zip(a, b):
        different |= x ^ y
    return different == 0

Encode Using CBC_HMAC:

In [8]:
hmac_key = get_random_bytes(16)
plaintext = pad(data, AES.block_size)
iv_bytes = get_random_bytes(AES.block_size)
HMAC_cipher = AES.new(key, AES.MODE_CBC, iv_bytes)
encrypted_data = HMAC_cipher.encrypt(plaintext)
iv = iv_bytes + encrypted_data #secret prefix mac
signature = hmac.new(hmac_key, iv, hashlib.sha256).digest()
print('Encrypted Data:' + b16encode(encrypted_data).decode('utf-8'))
print('IV: ' + b16encode(iv_bytes).decode('utf-8'))
print('Signature: ' + b16encode(signature).decode('utf-8'))

Encrypted Data:544195CEDF2666C4D8726A5527B3D41447DA38EF10CFB99F7A8141341FDB851159D45D327B25A385D755B947DF27E65333273F05797580678012AF4BF9EE1CEA9ABA6EE9BD9BA18E1AAD00E3CB6C2DC85B3AF613BB9FC4EF503601418C925A12E37BFF56A6CDEA4BBFFC3C97BB3E358E707C5026920B580AEE235768F9C0D958
IV: 34A9D50FCA425EDC4D41F6560F000DB7
Signature: 1C49BEE47F37DD90E16DD2545E304CB73F82021C02481FDA8F08576B5A135AC3


# Tamper Attack:

In [13]:
iv = b16encode(iv_bytes).decode('utf-8')
ciphertext = b16encode(encrypted_data).decode('utf-8')
ct_json = json.dumps({'iv': iv, 'ct': ciphertext})
altered_ciphertext = attack(ct_json)
altered_ciphertext = b16decode(json.loads(altered_ciphertext)['ct'])
decrypt_message(altered_ciphertext, iv_bytes, signature, key, hmac_key)

Attack!


b'Accnt:0003336669Accnt:0123456789Descr:000000000000000000000000000000000000000000\x80\xe0\x9eV\x8e\xe7\xd8\x14\xb3\xe2:\n\x07=\x0e\x03Amount:923456789'

# Replay Attack:

In [15]:
altered_ciphertext = b"".join([encrypted_data[32:64], encrypted_data[0:32], encrypted_data[64:]])
decrypt_message(altered_ciphertext, iv_bytes, signature, key, hmac_key)

Attack!


b"7\x16\x9e\x83\xa8\xb7\xd7s\x07\xf0\x87R \xeb\xb8\x960000000000000000F\xed\x89d\xc7\r\xee\x8b\xfd`j.\xc0\xd8'dAccnt:0123456789D\xcd7\xdaY\x8a\t\xc8\xca\xa3\xdeO\xd6\x05\xa9\xcb0000000000000000Amount:023456789"